In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

data = pd.read_csv("https://raw.githubusercontent.com/amankharwal/Website-data/master/demand.csv")

print(data.head())
print(data.info())
print(data.describe())

data = data.drop(columns=['ID'])

print(data.isnull().sum())

data = data.dropna()

X = data[['Store ID', 'Total Price', 'Base Price']]
y = data['Units Sold']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = Sequential([
    Dense(64, input_dim=3, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1)
])

model.compile(optimizer=Adam(learning_rate=0.01), loss='mean_squared_error')

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model.fit(X_train_scaled, y_train, validation_split=0.2, epochs=25, batch_size=10, callbacks=[early_stopping])

loss = model.evaluate(X_test_scaled, y_test)
print(f'Test Loss: {loss}')

y_pred = model.predict(X_test_scaled)

def evaluate_model(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    return mse, mae, r2


mse, mae, r2 = evaluate_model(y_test, y_pred)
print(f"Deep Learning Model - MSE: {mse}, MAE: {mae}, R2: {r2}")

df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred.flatten()})


sns.scatterplot(x='Actual', y='Predicted', data=df)
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title('Neural Network Predictions')
plt.show()
